In [ ]:
pip install -q langchain openai faiss-cpu tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 30.8 MB/s eta 0:00:00


In [ ]:
import os
import requests
import time
import json
import random

from tqdm import tqdm
from collections import defaultdict
from requests.exceptions import ConnectionError

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
base_path = "/content/drive/MyDrive/chatbot_bps"
os.makedirs(base_path, exist_ok=True)


In [ ]:
save_folder = os.path.join(base_path, "data_json")
vector_folder = os.path.join(base_path, "vector_store")
domain_file = os.path.join(base_path, "domain_var_ids.json")
save_folder_all = os.path.join(base_path, "data_json_all")


In [ ]:
# Domain mapping (nama ke kode)
domain_mapping = {
    "Prov. Jawa Timur": "3500",
    "Kab. Bangkalan": "3526", "Kab. Banyuwangi": "3510", "Kab. Blitar": "3505",
    "Kab. Bojonegoro": "3522", "Kab. Bondowoso": "3511", "Kab. Gresik": "3525",
    "Kab. Jember": "3509", "Kab. Jombang": "3517", "Kab. Kediri": "3506",
    "Kab. Lamongan": "3524", "Kab. Lumajang": "3508", "Kab. Madiun": "3519",
    "Kab. Magetan": "3520", "Kab. Malang": "3507", "Kab. Mojokerto": "3516",
    "Kab. Nganjuk": "3518", "Kab. Ngawi": "3521", "Kab. Pacitan": "3501",
    "Kab. Pamekasan": "3528", "Kab. Pasuruan": "3514", "Kab. Ponorogo": "3502",
    "Kab. Probolinggo": "3513", "Kab. Sampang": "3527", "Kab. Sidoarjo": "3515",
    "Kab. Situbondo": "3512", "Kab. Sumenep": "3529", "Kab. Trenggalek": "3503",
    "Kab. Tuban": "3523", "Kab. Tulungagung": "3504", "Kota Batu": "3579",
    "Kota Blitar": "3572", "Kota Kediri": "3571", "Kota Madiun": "3577",
    "Kota Malang": "3573", "Kota Mojokerto": "3576", "Kota Pasuruan": "3575",
    "Kota Probolinggo": "3574", "Kota Surabaya": "3578"
}

API_KEY = "2ad01e6a21b015ea1ff8805ced02600c"

# # Folder simpan
# save_folder = "data_json"
# os.makedirs(save_folder, exist_ok=True)

# with open(domain_file, "w") as f:
#     json.dump(domain_var_ids, f)

# # Loop domain dan var_id
# for domain_name, domain_code in tqdm(domain_mapping.items(), desc="Fetching all data"):
#     var_ids = domain_var_ids.get(domain_code, [])  # ambil var_id untuk domain ini
#     for var_id in var_ids:
#         file_path = os.path.join(save_folder, f"{domain_code}_{var_id}.json")
#         if os.path.exists(file_path):
#             continue  # skip if already downloaded

#         url = f"https://webapi.bps.go.id/v1/api/list/model/data/domain/{domain_code}/var/{var_id}/key/{API_KEY}/"
#         try:
#             r = requests.get(url)
#             if r.status_code == 200:
#                 with open(file_path, "w", encoding="utf-8") as f:
#                     f.write(r.text)
#             else:
#                 print(f"[{domain_name}] var {var_id} - Failed ({r.status_code})")
#         except Exception as e:
#             print(f"Error fetching {domain_code}-{var_id}: {e}")
#         time.sleep(0.5)  # hindari rate limit


In [ ]:
class BPSVariableScraper:
    def __init__(self, api_key):
        self.base_url = "https://webapi.bps.go.id/v1/api"
        self.api_key = api_key
        self.headers = {"User-Agent": "BPS-Chatbot/1.0"}
        self.var_map = {}
        self.domain_vars = defaultdict(list)

    def get_variables_for_domain(self, domain):
        page = 1
        total_pages = 1
        variables = []

        while page <= total_pages:
            url = f"{self.base_url}/list/model/var/domain/{domain}/page/{page}/key/{self.api_key}/"
            r = requests.get(url, headers=self.headers)
            if r.status_code != 200:
                break
            data = r.json()
            if page == 1:
                total_pages = data["data"][0]["pages"]
            variables.extend(data["data"][1])
            page += 1
        return variables

    def get_all_variables_grouped_by_domain(self, domain_mapping):
        for domain_name, domain_code in tqdm(domain_mapping.items(), desc="Scraping Semua Domain"):
            domain_vars = self.get_variables_for_domain(domain_code)
            for var in domain_vars:
                var_id = var["var_id"]
                if var_id not in self.var_map:
                    self.var_map[var_id] = {
                        'title': var['title'],
                        'unit': var['unit'],
                        'subcategory': var['sub_name'],
                        'definition': var['def']
                    }
                self.domain_vars[domain_code].append(var_id)
        return dict(self.domain_vars)

# Inisialisasi scraper
scraper = BPSVariableScraper(API_KEY)

# Scrape var_id per domain
domain_var_ids = scraper.get_all_variables_grouped_by_domain(domain_mapping)

# Simpan hasilnya ke Google Drive (path dari domain_file)
with open(domain_file, "w", encoding="utf-8") as f:
    json.dump(domain_var_ids, f, indent=2, separators=(", ", ": "), ensure_ascii=False)

print(f"Total domain terdeteksi: {len(domain_var_ids)}")


Scraping Semua Domain:   3%|▎         | 1/39 [00:52<33:03, 52.19s/it]


KeyboardInterrupt: 

In [ ]:
MAX_RETRY = 3

# Load domain → var_id mapping
with open(domain_file, "r") as f:  # ✅
    domain_var_ids = json.load(f)

# Folder penyimpanan data (Google Drive)
save_folder = os.path.join(base_path, "data_json_all")
os.makedirs(save_folder, exist_ok=True)

# Loop domain dan var_id
for domain_code, var_ids in tqdm(domain_var_ids.items(), desc="🔁 Fetching all data"):
    for var_id in var_ids:
        filepath = os.path.join(save_folder, f"{domain_code}_{var_id}.json")
        if os.path.exists(filepath): continue

        url = f"https://webapi.bps.go.id/v1/api/list/model/data/domain/{domain_code}/var/{var_id}/key/{API_KEY}/"

        retry_count = 0
        while retry_count < MAX_RETRY:
            try:
                r = requests.get(url)
                if r.status_code == 200:
                    data = r.json()
                    with open(filepath, "w", encoding="utf-8") as f:
                        json.dump(data, f, indent=2)
                    break  # keluar dari loop retry
                else:
                    print(f"❌ Gagal: {domain_code}-{var_id} → Status {r.status_code}")
                    break
            except ConnectionError as e:
                retry_count += 1
                print(f"⚠️ Retry {retry_count} untuk {domain_code}-{var_id} (koneksi terputus)")
                time.sleep(1 + random.random())  # delay acak

        time.sleep(0.4)



🔁 Fetching all data: 100%|██████████| 39/39 [3:39:28<00:00, 337.66s/it]


In [ ]:
import os
import json

base_path = "/content/drive/MyDrive/chatbot_bps"
folder_path = os.path.join(base_path, "data_json")

total_files = 0
total_datapoints = 0

for filename in os.listdir(folder_path):
    if filename.endswith(".json") and filename.startswith("3500_"):
        total_files += 1
        with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                total_datapoints += len(data.get("datacontent", {}))
            except:
                pass  # skip file yang error

print(f"📁 Total file JSON: {total_files}")
print(f"📊 Total datacontent (nilai statistik): {total_datapoints}")


📁 Total file JSON: 415
📊 Total datacontent (nilai statistik): 151138
